<a href="https://colab.research.google.com/github/AgustinBatistelli/programacion_concurrente/blob/master/TP3/Python/TP1_P3_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Se crea archivo con un poco de texto

In [ ]:
!echo "Lorem Ipsum es simplemente el texto de relleno de las imprentas y archivos de texto. Lorem Ipsum ha sido el texto de relleno estándar de las industrias desde el año 1500, cuando un impresor (N. del T. persona que se dedica a la imprenta) desconocido usó una galería de textos y los mezcló de tal manera que logró hacer un libro de textos especimen. No sólo sobrevivió 500 años, sino que tambien ingresó como texto de relleno en documentos electrónicos, quedando esencialmente igual al original. Fue popularizado en los 60s con la creación de las hojas \"Letraset\", las cuales contenian pasajes de Lorem Ipsum, y más recientemente con software de autoedición, como por ejemplo Aldus PageMaker, el cual incluye versiones de Lorem Ipsum." > archivo.txt

Programa para copiar el archivo usando pipes

In [ ]:
%%writefile copy_copy.py
import multiprocessing
import pathlib
import time
import sys

def send_file(write_end, file_name, chunk_size):
  try:
    with open(file_name, "rb") as file:
      bytes = file.read(chunk_size)
      while bytes:
        print(bytes)
        write_end.send_bytes(bytes)
        bytes = file.read(chunk_size)
    write_end.close()
  except FileNotFoundError:
    print("Archivo no encontrado.")

def child(read_end, origin_file, chunk_size = 512):
  # create destiny file
  destiny_file = origin_file.with_stem(origin_file.stem + '_copy')
  file_content = b''
  counter = 0
  # get chunks to receive
  times = origin_file.stat().st_size / chunk_size
  try:
    with open(destiny_file, 'wb') as file:
      while counter < times:
        msg = read_end.recv_bytes()
        file.write(msg)
        counter += 1
  except EOFError:
    print("Algo salió mal con el pipe.")
  read_end.close()

def father(file_name, chunk_size = 512):
  # Make pipe
  read_end, write_end = multiprocessing.Pipe()
  # Start child
  process = multiprocessing.Process(target=child, args=(read_end, file_name, chunk_size, ))
  process.start()
  send_file(write_end, file_name, chunk_size)
  process.join()

if __name__ == '__main__':
    args = sys.argv
    chunk_size = None
    print(args)
    if len(args) < 2:
      print("Debe pasar la ruta de un archivo válido.")
      exit(-1)
    try:
      if len(args) > 2 and int(args[2]) > 0:
        chunk_size = int(args[2])
      else:
        print("El tamaño del chunk debe ser mayor a 0 bytes")
    except ValueError:
      print("El tamaño del chunk debe ser un número natural medido en bytes.")

    #Check file
    file_name = pathlib.Path(args[1])
    if not file_name.exists() or not file_name.is_file():
      print("Archivo no encontrado")
      exit -1
    if chunk_size == None:
      father(file_name)
    else:
      father(file_name, chunk_size)

Writing copy_copy.py


Listo archivos para segurarme que tengo creados tanto el archivo a copiar como el script de python

In [ ]:
!ls -lash 

total 40K
4.0K drwxr-xr-x 1 root root 4.0K Apr 30 13:10 .
4.0K drwxr-xr-x 1 root root 4.0K Apr 30 13:07 ..
4.0K -rw-r--r-- 1 root root  748 Apr 30 13:15 archivo.txt
4.0K drwxr-xr-x 4 root root 4.0K Apr 27 13:34 .config
4.0K -rw-r--r-- 1 root root 1.9K Apr 30 13:15 copy_copy.py
4.0K -rw-r--r-- 1 root root  166 Apr 30 13:16 errors
4.0K -rw-r--r-- 1 root root  106 Apr 30 13:09 salida
4.0K -rw-r--r-- 1 root root   58 Apr 30 13:16 salida3
4.0K -rw-r--r-- 1 root root   58 Apr 30 13:10 salida4
4.0K drwxr-xr-x 1 root root 4.0K Apr 27 13:35 sample_data


Para ejecutar el programa sin bloquear la consola. El programa espera, si o si recibir un nombre de archivo como parametro o dará error y finalizará.

Opcionalmente, se puede pasar un segundo parametro que debe ser un número natural, que representa la cantidad de bytes a pasar por chunk en el pipe, en caso que no se pase este parametro los chunks seran de 512 bytes.

In [ ]:
!nohup python copy_copy.py 'archivo.txt' -1024 1>salida 2>errors &

In [ ]:
!ls sample_data

anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [ ]:
!nohup python copy_copy.py 'archivo.txt' 1>salida2 2>errors &

In [ ]:
!cat salida2

['copy_copy.py', 'file.txt']
El tamaño del chunk debe ser mayor a 0 bytes
Archivo no encontrado


In [ ]:
!nohup python copy_copy.py 'archivo.txt' 256 1>salida3 2>errors &

In [ ]:
!cat salida3

['copy_copy.py', 'archivo.txt', '256']
b'Lorem Ipsum es simplemente el texto de relleno de las imprentas y archivos de texto. Lorem Ipsum ha sido el texto de relleno est\xc3\xa1ndar de las industrias desde el a\xc3\xb1o 1500, cuando un impresor (N. del T. persona que se dedica a la imprenta) desconocido us\xc3\xb3'
b' una galer\xc3\xada de textos y los mezcl\xc3\xb3 de tal manera que logr\xc3\xb3 hacer un libro de textos especimen. No s\xc3\xb3lo sobrevivi\xc3\xb3 500 a\xc3\xb1os, sino que tambien ingres\xc3\xb3 como texto de relleno en documentos electr\xc3\xb3nicos, quedando esencialmente igual al original. Fue po'
b'pularizado en los 60s con la creaci\xc3\xb3n de las hojas "Letraset", las cuales contenian pasajes de Lorem Ipsum, y m\xc3\xa1s recientemente con software de autoedici\xc3\xb3n, como por ejemplo Aldus PageMaker, el cual incluye versiones de Lorem Ipsum.\n'


In [ ]:
!nohup python copy_copy.py 'archivo.txt' 128 1>salida4 2>errors &

In [ ]:
!cat salida4

['copy_copy.py', 'archivo.txt', '128']
b'Lorem Ipsum es simplemente el texto de relleno de las imprentas y archivos de texto. Lorem Ipsum ha sido el texto de relleno est'
b'\xc3\xa1ndar de las industrias desde el a\xc3\xb1o 1500, cuando un impresor (N. del T. persona que se dedica a la imprenta) desconocido us\xc3\xb3'
b' una galer\xc3\xada de textos y los mezcl\xc3\xb3 de tal manera que logr\xc3\xb3 hacer un libro de textos especimen. No s\xc3\xb3lo sobrevivi\xc3\xb3 500 a\xc3\xb1o'
b's, sino que tambien ingres\xc3\xb3 como texto de relleno en documentos electr\xc3\xb3nicos, quedando esencialmente igual al original. Fue po'
b'pularizado en los 60s con la creaci\xc3\xb3n de las hojas "Letraset", las cuales contenian pasajes de Lorem Ipsum, y m\xc3\xa1s recientement'
b'e con software de autoedici\xc3\xb3n, como por ejemplo Aldus PageMaker, el cual incluye versiones de Lorem Ipsum.\n'


In [ ]:
!cat archivo.txt

Lorem Ipsum es simplemente el texto de relleno de las imprentas y archivos de texto. Lorem Ipsum ha sido el texto de relleno estándar de las industrias desde el año 1500, cuando un impresor (N. del T. persona que se dedica a la imprenta) desconocido usó una galería de textos y los mezcló de tal manera que logró hacer un libro de textos especimen. No sólo sobrevivió 500 años, sino que tambien ingresó como texto de relleno en documentos electrónicos, quedando esencialmente igual al original. Fue popularizado en los 60s con la creación de las hojas "Letraset", las cuales contenian pasajes de Lorem Ipsum, y más recientemente con software de autoedición, como por ejemplo Aldus PageMaker, el cual incluye versiones de Lorem Ipsum.
